In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import pickle

class MovieRecommender:
    def __init__(self, movies):
        self.movies = movies
        self.titles = movies['title']
        self.indices = pd.Series(movies.index, index=movies['title'])
        
        # Initialize TF-IDF matrix based on genres
        self.cv = TfidfVectorizer()
        self.tfidf_matrix = self.cv.fit_transform(movies['genres'])
        
        # Compute cosine similarity matrix
        self.cosine_sim = linear_kernel(self.tfidf_matrix, self.tfidf_matrix)
    
    def get_similar_movies(self, title, top_n=10):
        if title not in self.indices:
            return "Movie not found"
        
        # Get the index of the movie
        idx = self.indices[title]
        
        # Pairwise cosine similarity scores
        sim_scores = list(enumerate(self.cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        
        # Get the indices of top_n similar movies
        sim_scores = sim_scores[1:top_n + 1]
        movie_indices = [i[0] for i in sim_scores]
        
        # Return the top_n movie titles
        return self.titles.iloc[movie_indices]

# Load movie data
movies = pd.read_csv('movies.csv')

# Initialize and save the model
recommender = MovieRecommender(movies)

with open("model.pkl", "wb") as file:
    pickle.dump(recommender, file)
